# * VINSIGHT : Revenue

## Parameter

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Import Transaction

In [2]:
''' Execute transaction '''


# Input parameter
v_start_date = 20240101
print(f'\nParameter input...')
print(f'   -> v_start_date: {v_start_date}')

curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nData as of {curr_datetime}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_MTH, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD, AREA_NAME
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL_SNAP END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL_SNAP END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL_SNAP END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL_SNAP END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL_SNAP END) HH
        , MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD IN (
        'B0R000100' --Total Revenue
        , 'DB0R000100' --Total Revenue : DTAC
        , 'TB0R000100' --Total Revenue : TRUE
        , 'B0R000100GEO' --Total Revenue (Geo)
        , 'DB0R000100GEO' --Total Revenue (Geo) : DTAC
        , 'TB0R000100GEO' --Total Revenue (Geo) : TRUE

        , 'B0R000101' --Mobile Revenue
        , 'DB0R000101' --Mobile Revenue : DTAC
        , 'TB0R000101' --Mobile Revenue : TMH
        , 'B0R000101GEO' --Mobile Revenue (Geo)
        , 'DB0R000101GEO' --Mobile Revenue (Geo) : DTAC
        , 'TB0R000101GEO' --Mobile Revenue (Geo) : TMH
        
        , 'B1R000100' --Prepaid Revenue
        , 'DB1R000100' --Prepaid Revenue : DTAC
        , 'TB1R000100' --Prepaid Revenue : TMH
         
        , 'B2R000100' --Postpaid Revenue
        , 'B2R010100' --Postpaid Revenue B2C
        , 'B2R020100' --Postpaid Revenue B2B
        , 'DB2R000100' --Postpaid Revenue : DTAC
        , 'DB2R010100' --Postpaid Revenue B2C : DTAC
        , 'DB2R020100' --Postpaid Revenue B2B : DTAC
        , 'TB2R000100' --Postpaid Revenue : TMH
        , 'TB2R010100' --Postpaid Revenue B2C : TMH
        , 'TB2R020100' --Postpaid Revenue B2B : TMH
        
        , 'TB3R000100' --TOL Revenue
        
        , 'TB4R000100' --TVS Revenue
        )
    AND AREA_TYPE IN ('C','P','G','H','HH')
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY TM_KEY_MTH, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
""")


try:
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    print(f'\nDataFrame: {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...
   -> v_start_date: 20240101

Data as of 2025-06-05, 10:24:37

DataFrame: 13536 rows, 13 columns


In [3]:
''' Automate Currently Period '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
prev_mth = str(chk_src_df['TM_KEY_MTH'].drop_duplicates().astype(int).shift().max())

## Daily (Corp & Geo)

In [4]:
''' Parameter '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = 202505

# Check Result
print(f'v_tm_key_mth: {v_tm_key_mth}')

v_tm_key_mth: 202505


In [5]:
''' Revenue(Corp) Daily '''

v_metric_list = [
    'B0R000100' #Total Revenue
    , 'B1R000100' #Prepaid Revenue
    # , 'DB1R000100' #Prepaid Revenue : DTAC
    # , 'TB1R000100' #Prepaid Revenue : TMH
    , 'B2R000100' #Postpaid Revenue
    # , 'DB2R000100' #Postpaid Revenue : DTAC
    # , 'TB2R000100' #Postpaid Revenue : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_corp_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C']].copy()
rev_corp_daily_df = rev_corp_daily_df.loc[rev_corp_daily_df['METRIC_CD'].isin(v_metric_list)]
rev_corp_daily_df = rev_corp_daily_df.loc[rev_corp_daily_df['TM_KEY_MTH']>=v_tm_key_mth]

rev_corp_daily_df['TOTAL(Corp)'] = np.where(rev_corp_daily_df['METRIC_CD']=='B0R000100', rev_corp_daily_df['C'], 0)
rev_corp_daily_df['PRE'] = np.where(rev_corp_daily_df['METRIC_CD']=='B1R000100', rev_corp_daily_df['C'], 0)
# rev_corp_daily_df['PRE_T'] = np.where(rev_corp_daily_df['METRIC_CD']=='TB1R000100', rev_corp_daily_df['C'], 0)
# rev_corp_daily_df['PRE_D'] = np.where(rev_corp_daily_df['METRIC_CD']=='DB1R000100', rev_corp_daily_df['C'], 0)
rev_corp_daily_df['POST'] = np.where(rev_corp_daily_df['METRIC_CD']=='B2R000100', rev_corp_daily_df['C'], 0)
# rev_corp_daily_df['POST_T'] = np.where(rev_corp_daily_df['METRIC_CD']=='TB2R000100', rev_corp_daily_df['C'], 0)
# rev_corp_daily_df['POST_D'] = np.where(rev_corp_daily_df['METRIC_CD']=='DB2R000100', rev_corp_daily_df['C'], 0)
rev_corp_daily_df['TOL'] = np.where(rev_corp_daily_df['METRIC_CD']=='TB3R000100', rev_corp_daily_df['C'], 0)
rev_corp_daily_df['TVS'] = np.where(rev_corp_daily_df['METRIC_CD']=='TB4R000100', rev_corp_daily_df['C'], 0)

rev_corp_daily_df = rev_corp_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'TOTAL(Corp)':'sum', 'PRE':'sum', 'POST':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_corp_daily_df = rev_corp_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY']).reset_index()
rev_corp_daily_df = rev_corp_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'TOTAL(Corp)', 'PRE', 'POST', 'TOL', 'TVS']]

mod_col_list = rev_corp_daily_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_corp_daily_df[col] = rev_corp_daily_df[col].apply(lambda x: format(x, ',.0f'))

rev_corp_daily_df

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,TOTAL(Corp),PRE,POST,TOL,TVS
0,202505,20250501,2025-06-05 03:21:13,"185,057,378","175,099,813","9,957,565",0,0
1,202505,20250502,2025-06-05 03:21:13,"1,260,298,454","162,930,833","841,261,243","221,415,792","34,690,587"
2,202505,20250503,2025-06-05 03:21:13,"237,078,095","146,752,230","90,325,865",0,0
3,202505,20250504,2025-06-05 03:21:13,"227,724,497","139,324,000","88,400,498",0,0
4,202505,20250505,2025-06-05 03:21:13,"230,532,922","144,754,994","85,777,928",0,0
5,202505,20250506,2025-06-05 03:21:13,"687,382,961","144,489,760","383,992,645","147,515,651","11,384,904"
6,202505,20250507,2025-06-05 03:21:13,"226,743,747","139,925,621","86,818,126",0,0
7,202505,20250508,2025-06-05 03:21:13,"227,799,230","141,534,062","86,265,167",0,0
8,202505,20250509,2025-06-05 03:21:13,"225,769,562","136,863,210","88,906,352",0,0
9,202505,20250510,2025-06-05 03:21:13,"980,474,262","137,704,908","578,292,257","233,926,371","30,550,725"


In [6]:
''' Revenue(Geo) Daily '''

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'B1R000100' #Prepaid Revenue
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'B2R010100' #Postpaid Revenue B2C
    , 'DB2R010100' #Postpaid Revenue B2C : DTAC
    , 'TB2R010100' #Postpaid Revenue B2C : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_geo_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
rev_geo_daily_df = rev_geo_daily_df.loc[rev_geo_daily_df['METRIC_CD'].isin(v_metric_list)]
rev_geo_daily_df = rev_geo_daily_df.loc[rev_geo_daily_df['TM_KEY_MTH']>=v_tm_key_mth]

rev_geo_daily_df['TOTAL(Geo)'] = np.where(rev_geo_daily_df['METRIC_CD']=='B0R000100GEO', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['PRE'] = np.where(rev_geo_daily_df['METRIC_CD']=='B1R000100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['PRE_T'] = np.where(rev_geo_daily_df['METRIC_CD']=='TB1R000100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['PRE_D'] = np.where(rev_geo_daily_df['METRIC_CD']=='DB1R000100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['POST_B2C'] = np.where(rev_geo_daily_df['METRIC_CD']=='B2R010100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['POST_B2C_T'] = np.where(rev_geo_daily_df['METRIC_CD']=='TB2R010100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['POST_B2C_D'] = np.where(rev_geo_daily_df['METRIC_CD']=='DB2R010100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['TOL'] = np.where(rev_geo_daily_df['METRIC_CD']=='TB3R000100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['TVS'] = np.where(rev_geo_daily_df['METRIC_CD']=='TB4R000100', rev_geo_daily_df['P'], 0)

rev_geo_daily_df = rev_geo_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'TOTAL(Geo)':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_geo_daily_df = rev_geo_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY']).reset_index()
rev_geo_daily_df = rev_geo_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'TOTAL(Geo)', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL', 'TVS']]

mod_col_list = rev_geo_daily_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_geo_daily_df[col] = rev_geo_daily_df[col].apply(lambda x: format(x, ',.0f'))

rev_geo_daily_df

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,TOTAL(Geo),PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL,TVS
0,202505,20250501,2025-06-05 03:21:13,"181,113,461","174,932,414","99,880,018","75,052,396","6,181,046",0,"6,181,046",0,0
1,202505,20250502,2025-06-05 03:21:13,"993,069,428","162,783,649","94,043,903","68,739,746","575,554,312","479,978,969","95,575,342","220,586,403","34,145,064"
2,202505,20250503,2025-06-05 03:21:13,"233,704,819","146,582,612","84,480,590","62,102,022","87,122,207",0,"87,122,207",0,0
3,202505,20250504,2025-06-05 03:21:13,"226,737,538","139,197,697","79,916,644","59,281,053","87,539,841",0,"87,539,841",0,0
4,202505,20250505,2025-06-05 03:21:13,"229,099,212","144,651,500","83,827,766","60,823,734","84,447,712",0,"84,447,712",0,0
5,202505,20250506,2025-06-05 03:21:13,"675,662,554","144,364,096","83,847,884","60,516,213","375,072,537","287,818,841","87,253,696","146,728,288","9,497,633"
6,202505,20250507,2025-06-05 03:21:13,"224,161,542","139,653,454","81,677,270","57,976,184","84,508,087",0,"84,508,087",0,0
7,202505,20250508,2025-06-05 03:21:13,"225,753,269","141,525,625","83,755,536","57,770,090","84,227,643",0,"84,227,643",0,0
8,202505,20250509,2025-06-05 03:21:13,"221,456,281","136,844,646","81,516,755","55,327,891","84,611,636",0,"84,611,636",0,0
9,202505,20250510,2025-06-05 03:21:13,"931,359,115","137,684,231","82,117,627","55,566,604","531,218,368","444,322,050","86,896,318","233,426,212","29,030,304"


## Prep Monthly Data

In [7]:
''' Monthly Summary '''

monthly_df = chk_src_df.copy()
monthly_df = monthly_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
monthly_df = monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()
monthly_df['DIFF (C-P)'] = monthly_df['C'] - monthly_df['P']

monthly_df_display = monthly_df.copy()
mod_col_list = monthly_df_display.iloc[:, 5:].columns.tolist()
for col in mod_col_list:
    monthly_df_display[col] = monthly_df_display[col].apply(lambda x: format(x, ',.0f'))

# monthly_df_display

In [8]:
# ''' Sample Data '''

# # sample_mth_df = monthly_df_display.loc[monthly_df_display['TM_KEY_MTH']==202501]
# # sample_mth_df

# sample_day_df = chk_src_df.loc[chk_src_df['TM_KEY_MTH']==202501].copy()
# sample_day_df.tail(3)

## Special Metric (Corp & Geo)

### Revenue Corporate (C)
    B0R000100	Total Revenue
    DB0R000100	Total Revenue : DTAC
    TB0R000100	Total Revenue : TRUE
    
    B0R000101	Mobile Revenue
    DB0R000101	Mobile Revenue : DTAC
    TB0R000101	Mobile Revenue : TMH

In [9]:
''' B0R000100 : Total Revenue '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('DB1R000100', 'TB1R000100', 'DB2R000100', 'TB2R000100', 'TB3R000100', 'TB4R000100', 'TNSC00147')

v_metric_list = [
    'B0R000100' #Total Revenue
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'DB2R000100' #Postpaid Revenue : DTAC
    , 'TB2R000100' #Postpaid Revenue : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

total_rev_corp_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C']].copy()
total_rev_corp_df = total_rev_corp_df.loc[total_rev_corp_df['METRIC_CD'].isin(v_metric_list)]

total_rev_corp_df['TOTAL'] = np.where(total_rev_corp_df['METRIC_CD']=='B0R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['PRE_T'] = np.where(total_rev_corp_df['METRIC_CD']=='TB1R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['PRE_D'] = np.where(total_rev_corp_df['METRIC_CD']=='DB1R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['POST_T'] = np.where(total_rev_corp_df['METRIC_CD']=='TB2R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['POST_D'] = np.where(total_rev_corp_df['METRIC_CD']=='DB2R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['TOL'] = np.where(total_rev_corp_df['METRIC_CD']=='TB3R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['TVS'] = np.where(total_rev_corp_df['METRIC_CD']=='TB4R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['CHK_SUM'] = np.where(total_rev_corp_df['METRIC_CD'].isin(['TB1R000100','DB1R000100','TB2R000100','DB2R000100','TB3R000100','TB4R000100']), total_rev_corp_df['C'], 0)

total_rev_corp_df = total_rev_corp_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_T':'sum', 'POST_D':'sum', 'TOL':'sum', 'TVS':'sum', 'CHK_SUM':'sum'})
total_rev_corp_df['CHK_DIFF'] = total_rev_corp_df['TOTAL'] - total_rev_corp_df['CHK_SUM']
total_rev_corp_df = total_rev_corp_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_rev_corp_df = total_rev_corp_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL', 'PRE_T', 'PRE_D', 'POST_T', 'POST_D', 'TOL', 'TVS']]

mod_col_list = total_rev_corp_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_rev_corp_df[col] = total_rev_corp_df[col].apply(lambda x: format(x, ',.0f'))
total_rev_corp_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL,PRE_T,PRE_D,POST_T,POST_D,TOL,TVS
0,202401,2025-06-05 03:21:13,"2,941,716,154","12,527,775,340","15,469,491,494","2,211,327,573","1,803,435,378","3,722,556,455","2,911,240,546","1,523,586,477","355,628,910"
1,202402,2025-06-05 03:21:13,"2,615,573,173","12,248,043,279","14,863,616,453","2,082,487,801","1,714,499,359","3,686,285,226","2,886,648,865","1,530,089,455","348,032,572"
2,202403,2025-06-05 03:21:13,"2,903,824,251","12,484,397,821","15,388,222,072","2,227,023,093","1,804,320,877","3,658,095,837","2,888,375,734","1,548,640,318","357,941,963"
3,202404,2025-06-05 03:21:13,"2,892,864,331","12,358,592,798","15,251,457,130","2,141,306,338","1,743,063,676","3,692,736,895","2,880,091,836","1,554,149,465","347,244,590"
4,202405,2025-06-05 03:21:13,"2,864,625,372","12,465,806,354","15,330,431,727","2,273,442,416","1,773,026,687","3,675,457,906","2,844,558,832","1,561,523,330","337,797,184"
5,202406,2025-06-05 03:21:13,"2,844,195,687","12,371,723,897","15,215,919,585","2,210,273,356","1,726,210,569","3,712,964,588","2,824,759,226","1,568,974,898","328,541,261"
6,202407,2025-06-05 03:21:13,"2,837,776,527","12,489,879,839","15,327,656,366","2,277,612,022","1,799,729,034","3,667,787,168","2,817,043,356","1,585,220,761","342,487,499"
7,202408,2025-06-05 03:21:13,"2,819,814,973","12,624,587,918","15,444,402,891","2,317,035,023","1,797,101,387","3,680,722,062","2,802,327,404","1,598,755,814","428,646,226"
8,202409,2025-06-05 03:21:13,"2,808,480,400","12,510,022,981","15,318,503,381","2,248,127,904","1,737,113,658","3,706,292,256","2,784,939,778","1,641,741,349","391,808,037"
9,202410,2025-06-05 03:21:13,"2,814,108,322","12,569,510,549","15,383,618,871","2,349,377,919","1,782,549,687","3,685,237,458","2,800,494,445","1,614,615,686","337,235,354"


In [10]:
''' TB0R000100 : Total Revenue : TRUE '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('TB1R000100', 'TB2R000100', 'TB3R000100', 'TB4R000100')

v_metric_list = [
    'TB0R000100' #Total Revenue : TRUE
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'TB2R000100' #Postpaid Revenue : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

total_rev_corp_true_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C']].copy()
total_rev_corp_true_df = total_rev_corp_true_df.loc[total_rev_corp_true_df['METRIC_CD'].isin(v_metric_list)]

total_rev_corp_true_df['TOTAL_T'] = np.where(total_rev_corp_true_df['METRIC_CD']=='TB0R000100', total_rev_corp_true_df['C'], 0)
total_rev_corp_true_df['PRE_T'] = np.where(total_rev_corp_true_df['METRIC_CD']=='TB1R000100', total_rev_corp_true_df['C'], 0)
total_rev_corp_true_df['POST_T'] = np.where(total_rev_corp_true_df['METRIC_CD']=='TB2R000100', total_rev_corp_true_df['C'], 0)
total_rev_corp_true_df['TOL'] = np.where(total_rev_corp_true_df['METRIC_CD']=='TB3R000100', total_rev_corp_true_df['C'], 0)
total_rev_corp_true_df['TVS'] = np.where(total_rev_corp_true_df['METRIC_CD']=='TB4R000100', total_rev_corp_true_df['C'], 0)
total_rev_corp_true_df['CHK_SUM'] = np.where(total_rev_corp_true_df['METRIC_CD'].isin(['TB1R000100','TB2R000100','TB3R000100','TB4R000100']), total_rev_corp_true_df['C'], 0)

total_rev_corp_true_df = total_rev_corp_true_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL_T':'sum', 'PRE_T':'sum', 'POST_T':'sum', 'TOL':'sum', 'TVS':'sum', 'CHK_SUM':'sum'})
total_rev_corp_true_df['CHK_DIFF'] = total_rev_corp_true_df['TOTAL_T'] - total_rev_corp_true_df['CHK_SUM']
total_rev_corp_true_df = total_rev_corp_true_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_rev_corp_true_df = total_rev_corp_true_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL_T', 'PRE_T', 'POST_T', 'TOL', 'TVS']]

mod_col_list = total_rev_corp_true_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_rev_corp_true_df[col] = total_rev_corp_true_df[col].apply(lambda x: format(x, ',.0f'))
total_rev_corp_true_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL_T,PRE_T,POST_T,TOL,TVS
0,202401,2025-06-05 03:21:13,0,"7,813,099,416","7,813,099,416","2,211,327,573","3,722,556,455","1,523,586,477","355,628,910"
1,202402,2025-06-05 03:21:13,-0,"7,646,895,055","7,646,895,055","2,082,487,801","3,686,285,226","1,530,089,455","348,032,572"
2,202403,2025-06-05 03:21:13,0,"7,791,701,211","7,791,701,211","2,227,023,093","3,658,095,837","1,548,640,318","357,941,963"
3,202404,2025-06-05 03:21:13,0,"7,735,437,287","7,735,437,287","2,141,306,338","3,692,736,895","1,554,149,465","347,244,590"
4,202405,2025-06-05 03:21:13,0,"7,848,220,835","7,848,220,835","2,273,442,416","3,675,457,906","1,561,523,330","337,797,184"
5,202406,2025-06-05 03:21:13,0,"7,820,754,103","7,820,754,103","2,210,273,356","3,712,964,588","1,568,974,898","328,541,261"
6,202407,2025-06-05 03:21:13,-0,"7,873,107,450","7,873,107,450","2,277,612,022","3,667,787,168","1,585,220,761","342,487,499"
7,202408,2025-06-05 03:21:13,0,"8,025,159,126","8,025,159,126","2,317,035,023","3,680,722,062","1,598,755,814","428,646,226"
8,202409,2025-06-05 03:21:13,0,"7,987,969,545","7,987,969,545","2,248,127,904","3,706,292,256","1,641,741,349","391,808,037"
9,202410,2025-06-05 03:21:13,0,"7,986,466,416","7,986,466,416","2,349,377,919","3,685,237,458","1,614,615,686","337,235,354"


In [11]:
''' DB0R000100 : Total Revenue : DTAC '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('DB1R000100', 'DB2R000100')

v_metric_list = [
    'DB0R000100' #Total Revenue : DTAC
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'DB2R000100' #Postpaid Revenue : DTAC
    ]

total_rev_corp_dtac_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C']].copy()
total_rev_corp_dtac_df = total_rev_corp_dtac_df.loc[total_rev_corp_dtac_df['METRIC_CD'].isin(v_metric_list)]

total_rev_corp_dtac_df['TOTAL_D'] = np.where(total_rev_corp_dtac_df['METRIC_CD']=='DB0R000100', total_rev_corp_dtac_df['C'], 0)
total_rev_corp_dtac_df['PRE_D'] = np.where(total_rev_corp_dtac_df['METRIC_CD']=='DB1R000100', total_rev_corp_dtac_df['C'], 0)
total_rev_corp_dtac_df['POST_D'] = np.where(total_rev_corp_dtac_df['METRIC_CD']=='DB2R000100', total_rev_corp_dtac_df['C'], 0)
total_rev_corp_dtac_df['CHK_SUM'] = np.where(total_rev_corp_dtac_df['METRIC_CD'].isin(['DB1R000100','DB2R000100']), total_rev_corp_dtac_df['C'], 0)

total_rev_corp_dtac_df = total_rev_corp_dtac_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL_D':'sum', 'PRE_D':'sum', 'POST_D':'sum', 'CHK_SUM':'sum'})
total_rev_corp_dtac_df['CHK_DIFF'] = total_rev_corp_dtac_df['TOTAL_D'] - total_rev_corp_dtac_df['CHK_SUM']
total_rev_corp_dtac_df = total_rev_corp_dtac_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_rev_corp_dtac_df = total_rev_corp_dtac_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL_D', 'PRE_D', 'POST_D']]

mod_col_list = total_rev_corp_dtac_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_rev_corp_dtac_df[col] = total_rev_corp_dtac_df[col].apply(lambda x: format(x, ',.0f'))
total_rev_corp_dtac_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL_D,PRE_D,POST_D
0,202401,2025-06-05 03:21:13,"2,941,716,154","4,714,675,924","7,656,392,078","1,803,435,378","2,911,240,546"
1,202402,2025-06-05 03:21:13,"2,615,573,173","4,601,148,224","7,216,721,397","1,714,499,359","2,886,648,865"
2,202403,2025-06-05 03:21:13,"2,903,824,251","4,692,696,611","7,596,520,862","1,804,320,877","2,888,375,734"
3,202404,2025-06-05 03:21:13,"2,892,864,331","4,623,155,511","7,516,019,842","1,743,063,676","2,880,091,836"
4,202405,2025-06-05 03:21:13,"2,864,625,372","4,617,585,519","7,482,210,892","1,773,026,687","2,844,558,832"
5,202406,2025-06-05 03:21:13,"2,844,195,687","4,550,969,794","7,395,165,481","1,726,210,569","2,824,759,226"
6,202407,2025-06-05 03:21:13,"2,837,776,527","4,616,772,389","7,454,548,916","1,799,729,034","2,817,043,356"
7,202408,2025-06-05 03:21:13,"2,819,814,973","4,599,428,792","7,419,243,764","1,797,101,387","2,802,327,404"
8,202409,2025-06-05 03:21:13,"2,808,480,400","4,522,053,436","7,330,533,835","1,737,113,658","2,784,939,778"
9,202410,2025-06-05 03:21:13,"2,814,108,322","4,583,044,133","7,397,152,454","1,782,549,687","2,800,494,445"


In [12]:
''' B0R000101 : Mobile Revenue '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('DB2R000100', 'TB2R000100', 'DB1R000100', 'TB1R000100')

v_metric_list = [
    'B0R000101' #Mobile Revenue
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'DB2R000100' #Postpaid Revenue : DTAC
    , 'TB2R000100' #Postpaid Revenue : TMH
    ]

mobile_rev_corp_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C']].copy()
mobile_rev_corp_df = mobile_rev_corp_df.loc[mobile_rev_corp_df['METRIC_CD'].isin(v_metric_list)]

mobile_rev_corp_df['MOBILE'] = np.where(mobile_rev_corp_df['METRIC_CD']=='B0R000101', mobile_rev_corp_df['C'], 0)
mobile_rev_corp_df['PRE_T'] = np.where(mobile_rev_corp_df['METRIC_CD']=='TB1R000100', mobile_rev_corp_df['C'], 0)
mobile_rev_corp_df['PRE_D'] = np.where(mobile_rev_corp_df['METRIC_CD']=='DB1R000100', mobile_rev_corp_df['C'], 0)
mobile_rev_corp_df['POST_T'] = np.where(mobile_rev_corp_df['METRIC_CD']=='TB2R000100', mobile_rev_corp_df['C'], 0)
mobile_rev_corp_df['POST_D'] = np.where(mobile_rev_corp_df['METRIC_CD']=='DB2R000100', mobile_rev_corp_df['C'], 0)
mobile_rev_corp_df['CHK_SUM'] = np.where(mobile_rev_corp_df['METRIC_CD'].isin(['TB1R000100','DB1R000100','TB2R000100','DB2R000100']), mobile_rev_corp_df['C'], 0)

mobile_rev_corp_df = mobile_rev_corp_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'MOBILE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_T':'sum', 'POST_D':'sum', 'CHK_SUM':'sum'})
mobile_rev_corp_df['CHK_DIFF'] = mobile_rev_corp_df['MOBILE'] - mobile_rev_corp_df['CHK_SUM']
mobile_rev_corp_df = mobile_rev_corp_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
mobile_rev_corp_df = mobile_rev_corp_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'MOBILE', 'PRE_T', 'PRE_D', 'POST_T', 'POST_D']]

mod_col_list = mobile_rev_corp_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    mobile_rev_corp_df[col] = mobile_rev_corp_df[col].apply(lambda x: format(x, ',.0f'))
mobile_rev_corp_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,MOBILE,PRE_T,PRE_D,POST_T,POST_D
0,202401,2025-06-05 03:21:13,"2,941,716,154","10,648,559,952","13,590,276,107","2,211,327,573","1,803,435,378","3,722,556,455","2,911,240,546"
1,202402,2025-06-05 03:21:13,"2,615,573,173","10,369,921,251","12,985,494,425","2,082,487,801","1,714,499,359","3,686,285,226","2,886,648,865"
2,202403,2025-06-05 03:21:13,"2,903,824,251","10,577,815,541","13,481,639,792","2,227,023,093","1,804,320,877","3,658,095,837","2,888,375,734"
3,202404,2025-06-05 03:21:13,"2,892,864,331","10,457,198,744","13,350,063,075","2,141,306,338","1,743,063,676","3,692,736,895","2,880,091,836"
4,202405,2025-06-05 03:21:13,"2,864,625,372","10,566,485,841","13,431,111,214","2,273,442,416","1,773,026,687","3,675,457,906","2,844,558,832"
5,202406,2025-06-05 03:21:13,"2,844,195,687","10,474,207,738","13,318,403,425","2,210,273,356","1,726,210,569","3,712,964,588","2,824,759,226"
6,202407,2025-06-05 03:21:13,"2,837,776,527","10,562,171,579","13,399,948,106","2,277,612,022","1,799,729,034","3,667,787,168","2,817,043,356"
7,202408,2025-06-05 03:21:13,"2,819,814,973","10,597,185,877","13,417,000,850","2,317,035,023","1,797,101,387","3,680,722,062","2,802,327,404"
8,202409,2025-06-05 03:21:13,"2,808,480,400","10,476,473,596","13,284,953,995","2,248,127,904","1,737,113,658","3,706,292,256","2,784,939,778"
9,202410,2025-06-05 03:21:13,"2,814,108,322","10,617,659,509","13,431,767,830","2,349,377,919","1,782,549,687","3,685,237,458","2,800,494,445"


In [13]:
''' TB0R000101 : Mobile Revenue : TMH '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('TB2R000100', 'TB1R000100')

v_metric_list = [
    'TB0R000101' #Mobile Revenue : TMH
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'TB2R000100' #Postpaid Revenue : TMH
    ]

mobile_rev_corp_tmh_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C']].copy()
mobile_rev_corp_tmh_df = mobile_rev_corp_tmh_df.loc[mobile_rev_corp_tmh_df['METRIC_CD'].isin(v_metric_list)]

mobile_rev_corp_tmh_df['MOBILE_T'] = np.where(mobile_rev_corp_tmh_df['METRIC_CD']=='TB0R000101', mobile_rev_corp_tmh_df['C'], 0)
mobile_rev_corp_tmh_df['PRE_T'] = np.where(mobile_rev_corp_tmh_df['METRIC_CD']=='TB1R000100', mobile_rev_corp_tmh_df['C'], 0)
mobile_rev_corp_tmh_df['POST_T'] = np.where(mobile_rev_corp_tmh_df['METRIC_CD']=='TB2R000100', mobile_rev_corp_tmh_df['C'], 0)
mobile_rev_corp_tmh_df['CHK_SUM'] = np.where(mobile_rev_corp_tmh_df['METRIC_CD'].isin(['TB1R000100','TB2R000100']), mobile_rev_corp_tmh_df['C'], 0)

mobile_rev_corp_tmh_df = mobile_rev_corp_tmh_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'MOBILE_T':'sum', 'PRE_T':'sum', 'POST_T':'sum', 'CHK_SUM':'sum'})
mobile_rev_corp_tmh_df['CHK_DIFF'] = mobile_rev_corp_tmh_df['MOBILE_T'] - mobile_rev_corp_tmh_df['CHK_SUM']
mobile_rev_corp_tmh_df = mobile_rev_corp_tmh_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
mobile_rev_corp_tmh_df = mobile_rev_corp_tmh_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'MOBILE_T', 'PRE_T', 'POST_T']]

mod_col_list = mobile_rev_corp_tmh_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    mobile_rev_corp_tmh_df[col] = mobile_rev_corp_tmh_df[col].apply(lambda x: format(x, ',.0f'))
mobile_rev_corp_tmh_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,MOBILE_T,PRE_T,POST_T
0,202401,2025-06-05 03:21:13,0,"5,933,884,028","5,933,884,028","2,211,327,573","3,722,556,455"
1,202402,2025-06-05 03:21:13,0,"5,768,773,028","5,768,773,028","2,082,487,801","3,686,285,226"
2,202403,2025-06-05 03:21:13,0,"5,885,118,930","5,885,118,930","2,227,023,093","3,658,095,837"
3,202404,2025-06-05 03:21:13,0,"5,834,043,232","5,834,043,232","2,141,306,338","3,692,736,895"
4,202405,2025-06-05 03:21:13,0,"5,948,900,322","5,948,900,322","2,273,442,416","3,675,457,906"
5,202406,2025-06-05 03:21:13,0,"5,923,237,944","5,923,237,944","2,210,273,356","3,712,964,588"
6,202407,2025-06-05 03:21:13,0,"5,945,399,190","5,945,399,190","2,277,612,022","3,667,787,168"
7,202408,2025-06-05 03:21:13,0,"5,997,757,085","5,997,757,085","2,317,035,023","3,680,722,062"
8,202409,2025-06-05 03:21:13,0,"5,954,420,160","5,954,420,160","2,248,127,904","3,706,292,256"
9,202410,2025-06-05 03:21:13,0,"6,034,615,376","6,034,615,376","2,349,377,919","3,685,237,458"


In [14]:
''' DB0R000101 : Mobile Revenue : DTAC '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('DB2R000100', 'DB1R000100')

v_metric_list = [
    'DB0R000101' #Mobile Revenue : DTAC
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'DB2R000100' #Postpaid Revenue : DTAC
    ]

mobile_rev_corp_dtac_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C']].copy()
mobile_rev_corp_dtac_df = mobile_rev_corp_dtac_df.loc[mobile_rev_corp_dtac_df['METRIC_CD'].isin(v_metric_list)]

mobile_rev_corp_dtac_df['MOBILE_D'] = np.where(mobile_rev_corp_dtac_df['METRIC_CD']=='DB0R000101', mobile_rev_corp_dtac_df['C'], 0)
mobile_rev_corp_dtac_df['PRE_D'] = np.where(mobile_rev_corp_dtac_df['METRIC_CD']=='DB1R000100', mobile_rev_corp_dtac_df['C'], 0)
mobile_rev_corp_dtac_df['POST_D'] = np.where(mobile_rev_corp_dtac_df['METRIC_CD']=='DB2R000100', mobile_rev_corp_dtac_df['C'], 0)
mobile_rev_corp_dtac_df['CHK_SUM'] = np.where(mobile_rev_corp_dtac_df['METRIC_CD'].isin(['DB1R000100','DB2R000100']), mobile_rev_corp_dtac_df['C'], 0)

mobile_rev_corp_dtac_df = mobile_rev_corp_dtac_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'MOBILE_D':'sum', 'PRE_D':'sum', 'POST_D':'sum', 'CHK_SUM':'sum'})
mobile_rev_corp_dtac_df['CHK_DIFF'] = mobile_rev_corp_dtac_df['MOBILE_D'] - mobile_rev_corp_dtac_df['CHK_SUM']
mobile_rev_corp_dtac_df = mobile_rev_corp_dtac_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
mobile_rev_corp_dtac_df = mobile_rev_corp_dtac_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'MOBILE_D', 'PRE_D', 'POST_D']]

mod_col_list = mobile_rev_corp_dtac_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    mobile_rev_corp_dtac_df[col] = mobile_rev_corp_dtac_df[col].apply(lambda x: format(x, ',.0f'))
mobile_rev_corp_dtac_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,MOBILE_D,PRE_D,POST_D
0,202401,2025-06-05 03:21:13,"2,941,716,154","4,714,675,924","7,656,392,078","1,803,435,378","2,911,240,546"
1,202402,2025-06-05 03:21:13,"2,615,573,173","4,601,148,224","7,216,721,397","1,714,499,359","2,886,648,865"
2,202403,2025-06-05 03:21:13,"2,903,824,251","4,692,696,611","7,596,520,862","1,804,320,877","2,888,375,734"
3,202404,2025-06-05 03:21:13,"2,892,864,331","4,623,155,511","7,516,019,842","1,743,063,676","2,880,091,836"
4,202405,2025-06-05 03:21:13,"2,864,625,372","4,617,585,519","7,482,210,892","1,773,026,687","2,844,558,832"
5,202406,2025-06-05 03:21:13,"2,844,195,687","4,550,969,794","7,395,165,481","1,726,210,569","2,824,759,226"
6,202407,2025-06-05 03:21:13,"2,837,776,527","4,616,772,389","7,454,548,916","1,799,729,034","2,817,043,356"
7,202408,2025-06-05 03:21:13,"2,819,814,973","4,599,428,792","7,419,243,764","1,797,101,387","2,802,327,404"
8,202409,2025-06-05 03:21:13,"2,808,480,400","4,522,053,436","7,330,533,835","1,737,113,658","2,784,939,778"
9,202410,2025-06-05 03:21:13,"2,814,108,322","4,583,044,133","7,397,152,454","1,782,549,687","2,800,494,445"


### Revenue Geo (P)
    B0R000100GEO	Total Revenue (Geo)
    DB0R000100GEO	Total Revenue (Geo) : DTAC
    TB0R000100GEO	Total Revenue (Geo) : TRUE
    
    B0R000101GEO	Mobile Revenue (Geo)
    DB0R000101GEO	Mobile Revenue (Geo) : DTAC
    TB0R000101GEO	Mobile Revenue (Geo) : TMH

In [15]:
''' B0R000100GEO : Total Revenue (Geo) '''

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'DB2R010100' #Postpaid Revenue B2C : DTAC
    , 'TB2R010100' #Postpaid Revenue B2C : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

total_rev_geo_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_rev_geo_df = total_rev_geo_df.loc[total_rev_geo_df['METRIC_CD'].isin(v_metric_list)]

total_rev_geo_df['TOTAL'] = np.where(total_rev_geo_df['METRIC_CD']=='B0R000100GEO', total_rev_geo_df['P'], 0)
total_rev_geo_df['PRE_T'] = np.where(total_rev_geo_df['METRIC_CD']=='TB1R000100', total_rev_geo_df['P'], 0)
total_rev_geo_df['PRE_D'] = np.where(total_rev_geo_df['METRIC_CD']=='DB1R000100', total_rev_geo_df['P'], 0)
total_rev_geo_df['POST_B2C_T'] = np.where(total_rev_geo_df['METRIC_CD']=='TB2R010100', total_rev_geo_df['P'], 0)
total_rev_geo_df['POST_B2C_D'] = np.where(total_rev_geo_df['METRIC_CD']=='DB2R010100', total_rev_geo_df['P'], 0)
total_rev_geo_df['TOL'] = np.where(total_rev_geo_df['METRIC_CD']=='TB3R000100', total_rev_geo_df['P'], 0)
total_rev_geo_df['TVS'] = np.where(total_rev_geo_df['METRIC_CD']=='TB4R000100', total_rev_geo_df['P'], 0)
total_rev_geo_df['CHK_SUM'] = np.where(total_rev_geo_df['METRIC_CD'].isin(['TB1R000100','DB1R000100','TB2R010100','DB2R010100','TB3R000100','TB4R000100']), total_rev_geo_df['P'], 0)

total_rev_geo_df = total_rev_geo_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL':'sum', 'TVS':'sum', 'CHK_SUM':'sum'})
total_rev_geo_df['CHK_DIFF'] = total_rev_geo_df['TOTAL'] - total_rev_geo_df['CHK_SUM']
total_rev_geo_df = total_rev_geo_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_rev_geo_df = total_rev_geo_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL', 'PRE_T', 'PRE_D', 'POST_B2C_T', 'POST_B2C_D', 'TOL', 'TVS']]

mod_col_list = total_rev_geo_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_rev_geo_df[col] = total_rev_geo_df[col].apply(lambda x: format(x, ',.0f'))
total_rev_geo_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL,PRE_T,PRE_D,POST_B2C_T,POST_B2C_D,TOL,TVS
0,202401,2025-06-05 03:21:13,0,"8,570,448,367","8,570,448,367","2,211,126,641","1,803,547,883",0,"2,709,538,268","1,507,069,397","339,166,179"
1,202402,2025-06-05 03:21:13,0,"8,341,491,265","8,341,491,265","2,082,104,053","1,714,609,789",0,"2,689,888,663","1,523,039,176","331,849,584"
2,202403,2025-06-05 03:21:13,0,"8,599,542,121","8,599,542,121","2,226,380,294","1,804,408,125",0,"2,686,745,365","1,539,811,431","342,196,905"
3,202404,2025-06-05 03:21:13,0,"8,443,300,426","8,443,300,426","2,140,030,223","1,743,154,862",0,"2,681,407,303","1,546,917,278","331,790,759"
4,202405,2025-06-05 03:21:13,0,"8,568,553,220","8,568,553,220","2,269,268,086","1,773,106,846",0,"2,648,304,046","1,554,971,583","322,902,658"
5,202406,2025-06-05 03:21:13,0,"8,434,228,011","8,434,228,011","2,206,174,307","1,726,295,968",0,"2,623,975,210","1,563,892,736","313,889,791"
6,202407,2025-06-05 03:21:13,0,"11,857,212,955","11,857,212,955","2,272,833,725","1,799,831,136","3,269,232,707","2,609,495,087","1,577,812,706","328,007,594"
7,202408,2025-06-05 03:21:13,-0,"11,992,892,504","11,992,892,504","2,312,752,271","1,797,206,658","3,283,314,789","2,597,627,729","1,592,482,287","409,508,769"
8,202409,2025-06-05 03:21:13,0,"11,856,710,578","11,856,710,578","2,245,163,825","1,737,197,388","3,281,480,710","2,586,882,098","1,633,490,490","372,496,067"
9,202410,2025-06-05 03:21:13,0,"11,900,384,524","11,900,384,524","2,345,529,988","1,782,644,560","3,272,186,553","2,590,323,955","1,598,256,224","311,443,243"


In [16]:
''' TB0R000100GEO : Total Revenue (Geo) : TRUE '''

v_metric_list = [
    'TB0R000100GEO' #Total Revenue (Geo) : TRUE
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'TB2R010100' #Postpaid Revenue B2C : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

total_rev_geo_true_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_rev_geo_true_df = total_rev_geo_true_df.loc[total_rev_geo_true_df['METRIC_CD'].isin(v_metric_list)]

total_rev_geo_true_df['TOTAL_T'] = np.where(total_rev_geo_true_df['METRIC_CD']=='TB0R000100GEO', total_rev_geo_true_df['P'], 0)
total_rev_geo_true_df['PRE_T'] = np.where(total_rev_geo_true_df['METRIC_CD']=='TB1R000100', total_rev_geo_true_df['P'], 0)
total_rev_geo_true_df['POST_B2C_T'] = np.where(total_rev_geo_true_df['METRIC_CD']=='TB2R010100', total_rev_geo_true_df['P'], 0)
total_rev_geo_true_df['TOL'] = np.where(total_rev_geo_true_df['METRIC_CD']=='TB3R000100', total_rev_geo_true_df['P'], 0)
total_rev_geo_true_df['TVS'] = np.where(total_rev_geo_true_df['METRIC_CD']=='TB4R000100', total_rev_geo_true_df['P'], 0)
total_rev_geo_true_df['CHK_SUM'] = np.where(total_rev_geo_true_df['METRIC_CD'].isin(['TB1R000100','TB2R010100','TB3R000100','TB4R000100']), total_rev_geo_true_df['P'], 0)

total_rev_geo_true_df = total_rev_geo_true_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL_T':'sum', 'PRE_T':'sum', 'POST_B2C_T':'sum', 'TOL':'sum', 'TVS':'sum', 'CHK_SUM':'sum'})
total_rev_geo_true_df['CHK_DIFF'] = total_rev_geo_true_df['TOTAL_T'] - total_rev_geo_true_df['CHK_SUM']
total_rev_geo_true_df = total_rev_geo_true_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_rev_geo_true_df = total_rev_geo_true_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL_T', 'PRE_T', 'POST_B2C_T', 'TOL', 'TVS']]

mod_col_list = total_rev_geo_true_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_rev_geo_true_df[col] = total_rev_geo_true_df[col].apply(lambda x: format(x, ',.0f'))
total_rev_geo_true_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL_T,PRE_T,POST_B2C_T,TOL,TVS
0,202401,2025-06-05 03:21:13,0,"4,057,362,216","4,057,362,216","2,211,126,641",0,"1,507,069,397","339,166,179"
1,202402,2025-06-05 03:21:13,0,"3,936,992,813","3,936,992,813","2,082,104,053",0,"1,523,039,176","331,849,584"
2,202403,2025-06-05 03:21:13,0,"4,108,388,630","4,108,388,630","2,226,380,294",0,"1,539,811,431","342,196,905"
3,202404,2025-06-05 03:21:13,0,"4,018,738,261","4,018,738,261","2,140,030,223",0,"1,546,917,278","331,790,759"
4,202405,2025-06-05 03:21:13,0,"4,147,142,328","4,147,142,328","2,269,268,086",0,"1,554,971,583","322,902,658"
5,202406,2025-06-05 03:21:13,0,"4,083,956,833","4,083,956,833","2,206,174,307",0,"1,563,892,736","313,889,791"
6,202407,2025-06-05 03:21:13,0,"7,447,886,733","7,447,886,733","2,272,833,725","3,269,232,707","1,577,812,706","328,007,594"
7,202408,2025-06-05 03:21:13,0,"7,598,058,116","7,598,058,116","2,312,752,271","3,283,314,789","1,592,482,287","409,508,769"
8,202409,2025-06-05 03:21:13,0,"7,532,631,092","7,532,631,092","2,245,163,825","3,281,480,710","1,633,490,490","372,496,067"
9,202410,2025-06-05 03:21:13,-0,"7,527,416,008","7,527,416,008","2,345,529,988","3,272,186,553","1,598,256,224","311,443,243"


In [17]:
''' DB0R000100GEO : Total Revenue (Geo) : DTAC '''

v_metric_list = [
    'DB0R000100GEO' #Total Revenue (Geo) : DTAC
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'DB2R010100' #Postpaid Revenue B2C : DTAC
    ]

total_rev_geo_dtac_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_rev_geo_dtac_df = total_rev_geo_dtac_df.loc[total_rev_geo_dtac_df['METRIC_CD'].isin(v_metric_list)]

total_rev_geo_dtac_df['TOTAL_D'] = np.where(total_rev_geo_dtac_df['METRIC_CD']=='DB0R000100GEO', total_rev_geo_dtac_df['P'], 0)
total_rev_geo_dtac_df['PRE_D'] = np.where(total_rev_geo_dtac_df['METRIC_CD']=='DB1R000100', total_rev_geo_dtac_df['P'], 0)
total_rev_geo_dtac_df['POST_B2C_D'] = np.where(total_rev_geo_dtac_df['METRIC_CD']=='DB2R010100', total_rev_geo_dtac_df['P'], 0)
total_rev_geo_dtac_df['CHK_SUM'] = np.where(total_rev_geo_dtac_df['METRIC_CD'].isin(['DB1R000100','DB2R010100']), total_rev_geo_dtac_df['P'], 0)

total_rev_geo_dtac_df = total_rev_geo_dtac_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL_D':'sum', 'PRE_D':'sum', 'POST_B2C_D':'sum', 'CHK_SUM':'sum'})
total_rev_geo_dtac_df['CHK_DIFF'] = total_rev_geo_dtac_df['TOTAL_D'] - total_rev_geo_dtac_df['CHK_SUM']
total_rev_geo_dtac_df = total_rev_geo_dtac_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_rev_geo_dtac_df = total_rev_geo_dtac_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL_D', 'PRE_D', 'POST_B2C_D']]

mod_col_list = total_rev_geo_dtac_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_rev_geo_dtac_df[col] = total_rev_geo_dtac_df[col].apply(lambda x: format(x, ',.0f'))
total_rev_geo_dtac_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL_D,PRE_D,POST_B2C_D
0,202401,2025-06-05 03:21:13,0,"4,513,086,151","4,513,086,151","1,803,547,883","2,709,538,268"
1,202402,2025-06-05 03:21:13,0,"4,404,498,452","4,404,498,452","1,714,609,789","2,689,888,663"
2,202403,2025-06-05 03:21:13,0,"4,491,153,491","4,491,153,491","1,804,408,125","2,686,745,365"
3,202404,2025-06-05 03:21:13,0,"4,424,562,165","4,424,562,165","1,743,154,862","2,681,407,303"
4,202405,2025-06-05 03:21:13,0,"4,421,410,892","4,421,410,892","1,773,106,846","2,648,304,046"
5,202406,2025-06-05 03:21:13,0,"4,350,271,177","4,350,271,177","1,726,295,968","2,623,975,210"
6,202407,2025-06-05 03:21:13,0,"4,409,326,223","4,409,326,223","1,799,831,136","2,609,495,087"
7,202408,2025-06-05 03:21:13,0,"4,394,834,388","4,394,834,388","1,797,206,658","2,597,627,729"
8,202409,2025-06-05 03:21:13,-0,"4,324,079,486","4,324,079,486","1,737,197,388","2,586,882,098"
9,202410,2025-06-05 03:21:13,0,"4,372,968,515","4,372,968,515","1,782,644,560","2,590,323,955"


In [18]:
''' B0R000101GEO : Mobile Revenue (Geo) '''

v_metric_list = [
    'B0R000101GEO' #Mobile Revenue (Geo)
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'DB2R010100' #Postpaid Revenue B2C : DTAC
    , 'TB2R010100' #Postpaid Revenue B2C : TMH
    ]

mobile_rev_geo_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
mobile_rev_geo_df = mobile_rev_geo_df.loc[mobile_rev_geo_df['METRIC_CD'].isin(v_metric_list)]

mobile_rev_geo_df['MOBILE'] = np.where(mobile_rev_geo_df['METRIC_CD']=='B0R000101GEO', mobile_rev_geo_df['P'], 0)
mobile_rev_geo_df['PRE_T'] = np.where(mobile_rev_geo_df['METRIC_CD']=='TB1R000100', mobile_rev_geo_df['P'], 0)
mobile_rev_geo_df['PRE_D'] = np.where(mobile_rev_geo_df['METRIC_CD']=='DB1R000100', mobile_rev_geo_df['P'], 0)
mobile_rev_geo_df['POST_B2C_T'] = np.where(mobile_rev_geo_df['METRIC_CD']=='TB2R010100', mobile_rev_geo_df['P'], 0)
mobile_rev_geo_df['POST_B2C_D'] = np.where(mobile_rev_geo_df['METRIC_CD']=='DB2R010100', mobile_rev_geo_df['P'], 0)
mobile_rev_geo_df['CHK_SUM'] = np.where(mobile_rev_geo_df['METRIC_CD'].isin(['TB1R000100','DB1R000100','TB2R010100','DB2R010100']), mobile_rev_geo_df['P'], 0)

mobile_rev_geo_df = mobile_rev_geo_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'MOBILE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'CHK_SUM':'sum'})
mobile_rev_geo_df['CHK_DIFF'] = mobile_rev_geo_df['MOBILE'] - mobile_rev_geo_df['CHK_SUM']
mobile_rev_geo_df = mobile_rev_geo_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
mobile_rev_geo_df = mobile_rev_geo_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'MOBILE', 'PRE_T', 'PRE_D', 'POST_B2C_T', 'POST_B2C_D']]

mod_col_list = mobile_rev_geo_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    mobile_rev_geo_df[col] = mobile_rev_geo_df[col].apply(lambda x: format(x, ',.0f'))
mobile_rev_geo_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,MOBILE,PRE_T,PRE_D,POST_B2C_T,POST_B2C_D
0,202401,2025-06-05 03:21:13,0,"6,724,212,792","6,724,212,792","2,211,126,641","1,803,547,883",0,"2,709,538,268"
1,202402,2025-06-05 03:21:13,0,"6,486,602,505","6,486,602,505","2,082,104,053","1,714,609,789",0,"2,689,888,663"
2,202403,2025-06-05 03:21:13,0,"6,717,533,785","6,717,533,785","2,226,380,294","1,804,408,125",0,"2,686,745,365"
3,202404,2025-06-05 03:21:13,0,"6,564,592,389","6,564,592,389","2,140,030,223","1,743,154,862",0,"2,681,407,303"
4,202405,2025-06-05 03:21:13,0,"6,690,678,979","6,690,678,979","2,269,268,086","1,773,106,846",0,"2,648,304,046"
5,202406,2025-06-05 03:21:13,0,"6,556,445,484","6,556,445,484","2,206,174,307","1,726,295,968",0,"2,623,975,210"
6,202407,2025-06-05 03:21:13,0,"9,951,392,655","9,951,392,655","2,272,833,725","1,799,831,136","3,269,232,707","2,609,495,087"
7,202408,2025-06-05 03:21:13,0,"9,990,901,448","9,990,901,448","2,312,752,271","1,797,206,658","3,283,314,789","2,597,627,729"
8,202409,2025-06-05 03:21:13,0,"9,850,724,021","9,850,724,021","2,245,163,825","1,737,197,388","3,281,480,710","2,586,882,098"
9,202410,2025-06-05 03:21:13,0,"9,990,685,057","9,990,685,057","2,345,529,988","1,782,644,560","3,272,186,553","2,590,323,955"


In [19]:
''' TB0R000101GEO : Mobile Revenue (Geo) : TMH '''

v_metric_list = [
    'TB0R000101GEO' #Mobile Revenue (Geo) : TMH
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'TB2R010100' #Postpaid Revenue B2C : TMH
    ]

mobile_rev_geo_tmh_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
mobile_rev_geo_tmh_df = mobile_rev_geo_tmh_df.loc[mobile_rev_geo_tmh_df['METRIC_CD'].isin(v_metric_list)]

mobile_rev_geo_tmh_df['MOBILE_T'] = np.where(mobile_rev_geo_tmh_df['METRIC_CD']=='TB0R000101GEO', mobile_rev_geo_tmh_df['P'], 0)
mobile_rev_geo_tmh_df['PRE_T'] = np.where(mobile_rev_geo_tmh_df['METRIC_CD']=='TB1R000100', mobile_rev_geo_tmh_df['P'], 0)
mobile_rev_geo_tmh_df['POST_B2C_T'] = np.where(mobile_rev_geo_tmh_df['METRIC_CD']=='TB2R010100', mobile_rev_geo_tmh_df['P'], 0)
mobile_rev_geo_tmh_df['CHK_SUM'] = np.where(mobile_rev_geo_tmh_df['METRIC_CD'].isin(['TB1R000100','TB2R010100']), mobile_rev_geo_tmh_df['P'], 0)

mobile_rev_geo_tmh_df = mobile_rev_geo_tmh_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'MOBILE_T':'sum', 'PRE_T':'sum', 'POST_B2C_T':'sum', 'CHK_SUM':'sum'})
mobile_rev_geo_tmh_df['CHK_DIFF'] = mobile_rev_geo_tmh_df['MOBILE_T'] - mobile_rev_geo_tmh_df['CHK_SUM']
mobile_rev_geo_tmh_df = mobile_rev_geo_tmh_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
mobile_rev_geo_tmh_df = mobile_rev_geo_tmh_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'MOBILE_T', 'PRE_T', 'POST_B2C_T']]

mod_col_list = mobile_rev_geo_tmh_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    mobile_rev_geo_tmh_df[col] = mobile_rev_geo_tmh_df[col].apply(lambda x: format(x, ',.0f'))
mobile_rev_geo_tmh_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,MOBILE_T,PRE_T,POST_B2C_T
0,202401,2025-06-05 03:21:13,0,"2,211,126,641","2,211,126,641","2,211,126,641",0
1,202402,2025-06-05 03:21:13,0,"2,082,104,053","2,082,104,053","2,082,104,053",0
2,202403,2025-06-05 03:21:13,0,"2,226,380,294","2,226,380,294","2,226,380,294",0
3,202404,2025-06-05 03:21:13,0,"2,140,030,223","2,140,030,223","2,140,030,223",0
4,202405,2025-06-05 03:21:13,0,"2,269,268,086","2,269,268,086","2,269,268,086",0
5,202406,2025-06-05 03:21:13,0,"2,206,174,307","2,206,174,307","2,206,174,307",0
6,202407,2025-06-05 03:21:13,0,"5,542,066,432","5,542,066,432","2,272,833,725","3,269,232,707"
7,202408,2025-06-05 03:21:13,0,"5,596,067,060","5,596,067,060","2,312,752,271","3,283,314,789"
8,202409,2025-06-05 03:21:13,0,"5,526,644,535","5,526,644,535","2,245,163,825","3,281,480,710"
9,202410,2025-06-05 03:21:13,0,"5,617,716,541","5,617,716,541","2,345,529,988","3,272,186,553"


In [20]:
''' DB0R000101GEO : Mobile Revenue (Geo) : DTAC '''

v_metric_list = [
    'DB0R000101GEO' #Mobile Revenue (Geo) : DTAC
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'DB2R010100' #Postpaid Revenue B2C : DTAC
    ]

mobile_rev_geo_dtac_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
mobile_rev_geo_dtac_df = mobile_rev_geo_dtac_df.loc[mobile_rev_geo_dtac_df['METRIC_CD'].isin(v_metric_list)]

mobile_rev_geo_dtac_df['MOBILE_D'] = np.where(mobile_rev_geo_dtac_df['METRIC_CD']=='DB0R000101GEO', mobile_rev_geo_dtac_df['P'], 0)
mobile_rev_geo_dtac_df['PRE_D'] = np.where(mobile_rev_geo_dtac_df['METRIC_CD']=='DB1R000100', mobile_rev_geo_dtac_df['P'], 0)
mobile_rev_geo_dtac_df['POST_B2C_D'] = np.where(mobile_rev_geo_dtac_df['METRIC_CD']=='DB2R010100', mobile_rev_geo_dtac_df['P'], 0)
mobile_rev_geo_dtac_df['CHK_SUM'] = np.where(mobile_rev_geo_dtac_df['METRIC_CD'].isin(['DB1R000100','DB2R010100']), mobile_rev_geo_dtac_df['P'], 0)

mobile_rev_geo_dtac_df = mobile_rev_geo_dtac_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'MOBILE_D':'sum', 'PRE_D':'sum', 'POST_B2C_D':'sum', 'CHK_SUM':'sum'})
mobile_rev_geo_dtac_df['CHK_DIFF'] = mobile_rev_geo_dtac_df['MOBILE_D'] - mobile_rev_geo_dtac_df['CHK_SUM']
mobile_rev_geo_dtac_df = mobile_rev_geo_dtac_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
mobile_rev_geo_dtac_df = mobile_rev_geo_dtac_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'MOBILE_D', 'PRE_D', 'POST_B2C_D']]

mod_col_list = mobile_rev_geo_dtac_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    mobile_rev_geo_dtac_df[col] = mobile_rev_geo_dtac_df[col].apply(lambda x: format(x, ',.0f'))
mobile_rev_geo_dtac_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,MOBILE_D,PRE_D,POST_B2C_D
0,202401,2025-06-05 03:21:13,0,"4,513,086,151","4,513,086,151","1,803,547,883","2,709,538,268"
1,202402,2025-06-05 03:21:13,0,"4,404,498,452","4,404,498,452","1,714,609,789","2,689,888,663"
2,202403,2025-06-05 03:21:13,0,"4,491,153,491","4,491,153,491","1,804,408,125","2,686,745,365"
3,202404,2025-06-05 03:21:13,0,"4,424,562,165","4,424,562,165","1,743,154,862","2,681,407,303"
4,202405,2025-06-05 03:21:13,0,"4,421,410,892","4,421,410,892","1,773,106,846","2,648,304,046"
5,202406,2025-06-05 03:21:13,0,"4,350,271,177","4,350,271,177","1,726,295,968","2,623,975,210"
6,202407,2025-06-05 03:21:13,0,"4,409,326,223","4,409,326,223","1,799,831,136","2,609,495,087"
7,202408,2025-06-05 03:21:13,0,"4,394,834,388","4,394,834,388","1,797,206,658","2,597,627,729"
8,202409,2025-06-05 03:21:13,-0,"4,324,079,486","4,324,079,486","1,737,197,388","2,586,882,098"
9,202410,2025-06-05 03:21:13,0,"4,372,968,515","4,372,968,515","1,782,644,560","2,590,323,955"


## Products Summary

In [21]:
''' Parameter '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = 202505

# Check Result
print(f'v_tm_key_mth: {v_tm_key_mth}')

v_tm_key_mth: 202505


### All Service

In [22]:
''' All Service (Corp) '''

v_product_grp = 'All Services'

all_service_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp].copy()
all_service_df = all_service_df.loc[~all_service_df['METRIC_CD'].str.contains('GEO')]
all_service_df = all_service_df.loc[all_service_df['TM_KEY_MTH']==v_tm_key_mth]
all_service_df = all_service_df.reset_index(drop=True)
all_service_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202505,All Services,B0R000100,Total Revenue,2025-06-04 23:49:33,"12,473,243,301","12,442,754,986",0,0,0,"30,488,315"
1,202505,All Services,DB0R000100,Total Revenue : DTAC,2025-06-04 23:49:33,"4,360,893,566","4,360,899,792",0,0,0,"-6,226"
2,202505,All Services,TB0R000100,Total Revenue : TRUE,2025-06-04 23:49:33,"8,112,349,735","8,081,855,194",0,0,0,"30,494,541"


In [23]:
''' All Service (Geo) '''

v_product_grp = 'All Services'

all_service_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp].copy()
all_service_geo_df = all_service_geo_df.loc[all_service_geo_df['METRIC_CD'].str.contains('GEO')]
all_service_geo_df = all_service_geo_df.loc[all_service_geo_df['TM_KEY_MTH']==v_tm_key_mth]
all_service_geo_df = all_service_geo_df.reset_index(drop=True)
all_service_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202505,All Services,B0R000100GEO,Total Revenue (Geo),2025-06-05 03:21:13,0,"11,871,407,370","11,693,922,189","11,693,520,514","11,693,461,930","-11,871,407,370"
1,202505,All Services,DB0R000100GEO,Total Revenue (Geo) : DTAC,2025-06-05 03:21:13,0,"4,159,705,332","4,159,703,780","4,159,614,985","4,159,614,985","-4,159,705,332"
2,202505,All Services,TB0R000100GEO,Total Revenue (Geo) : TRUE,2025-06-05 03:21:13,0,"7,711,702,038","7,534,218,408","7,533,905,529","7,533,846,945","-7,711,702,038"


### Mobile

In [24]:
''' Mobile (Corp) '''

v_product_grp = 'Mobile'

mobile_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
mobile_df = mobile_df.loc[~mobile_df['METRIC_CD'].str.contains('GEO')]
mobile_df = mobile_df.loc[mobile_df['TM_KEY_MTH']==v_tm_key_mth]
mobile_df = mobile_df.reset_index(drop=True)
mobile_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202505,Mobile,B0R000101,Mobile Revenue,2025-06-04 23:49:33,"10,550,624,069","10,536,363,812",0,0,0,"14,260,257"
1,202505,Mobile,DB0R000101,Mobile Revenue : DTAC,2025-06-04 23:49:33,"4,360,893,566","4,360,899,792",0,0,0,"-6,226"
2,202505,Mobile,TB0R000101,Mobile Revenue : TMH,2025-06-04 23:49:33,"6,189,730,503","6,175,464,020",0,0,0,"14,266,483"


In [25]:
''' Mobile (Geo) '''

v_product_grp = 'Mobile'

mobile_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
mobile_geo_df = mobile_geo_df.loc[mobile_geo_df['METRIC_CD'].str.contains('GEO')]
mobile_geo_df = mobile_geo_df.loc[mobile_geo_df['TM_KEY_MTH']==v_tm_key_mth]
mobile_geo_df = mobile_geo_df.reset_index(drop=True)
mobile_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202505,Mobile,B0R000101GEO,Mobile Revenue (Geo),2025-06-05 03:21:13,0,"9,965,016,196","9,787,531,014","9,786,913,839","9,786,913,839","-9,965,016,196"
1,202505,Mobile,DB0R000101GEO,Mobile Revenue (Geo) : DTAC,2025-06-05 03:21:13,0,"4,159,705,332","4,159,703,780","4,159,614,985","4,159,614,985","-4,159,705,332"
2,202505,Mobile,TB0R000101GEO,Mobile Revenue (Geo) : TMH,2025-06-05 03:21:13,0,"5,805,310,864","5,627,827,234","5,627,298,854","5,627,298,854","-5,805,310,864"


### Prepaid

In [26]:
''' Prepaid '''

v_product_grp = 'Prepaid'

prepaid_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
prepaid_df = prepaid_df.loc[prepaid_df['TM_KEY_MTH']==v_tm_key_mth]
prepaid_df = prepaid_df.reset_index(drop=True)
prepaid_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202505,Prepaid,B1R000100,Prepaid Revenue,2025-06-05 03:21:13,"3,941,403,668","3,927,715,673","3,750,441,071","3,749,838,379","3,749,838,379","13,687,996"
1,202505,Prepaid,DB1R000100,Prepaid Revenue : DTAC,2025-06-05 03:21:13,"1,662,576,972","1,662,583,198","1,662,583,198","1,662,505,359","1,662,505,359","-6,226"
2,202505,Prepaid,TB1R000100,Prepaid Revenue : TMH,2025-06-05 03:21:13,"2,474,647,604","2,460,381,121","2,282,909,616","2,282,383,708","2,282,383,708","14,266,483"


### Postpaid

In [27]:
''' Postpaid '''

v_product_grp = 'Postpaid'

postpaid_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_df = postpaid_df.loc[~postpaid_df['METRIC_NAME'].str.contains('B2C|B2B')]
postpaid_df = postpaid_df.loc[postpaid_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_df = postpaid_df.reset_index(drop=True)
postpaid_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202505,Postpaid,B2R000100,Postpaid Revenue,2025-06-04 23:49:33,"6,122,102,688","6,122,102,688",0,0,0,0
1,202505,Postpaid,DB2R000100,Postpaid Revenue : DTAC,2025-06-05 03:21:13,"2,698,316,594","2,698,316,594","2,698,315,042","2,697,221,058","2,697,221,058",0
2,202505,Postpaid,TB2R000100,Postpaid Revenue : TMH,2025-06-04 23:49:33,"3,715,082,899","3,715,082,899",0,0,0,0


In [28]:
''' Postpaid B2C '''

v_product_grp = 'Postpaid'

postpaid_b2c_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2c_df = postpaid_b2c_df.loc[postpaid_b2c_df['METRIC_NAME'].str.contains('B2C')]
postpaid_b2c_df = postpaid_b2c_df.loc[postpaid_b2c_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2c_df = postpaid_b2c_df.reset_index(drop=True)
postpaid_b2c_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202505,Postpaid,B2R010100,Postpaid Revenue B2C,2025-06-05 03:21:13,"5,574,765,425","5,574,765,425","5,574,751,750","5,574,740,756","5,574,740,756",0
1,202505,Postpaid,DB2R010100,Postpaid Revenue B2C : DTAC,2025-06-05 03:21:13,"2,497,122,134","2,497,122,134","2,497,120,582","2,497,109,626","2,497,109,626",0
2,202505,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-06-05 03:21:13,"3,344,929,743","3,344,929,743","3,344,917,619","3,344,915,146","3,344,915,146",0


In [29]:
''' Postpaid B2B '''

v_product_grp = 'Postpaid'

postpaid_b2b_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2b_df = postpaid_b2b_df.loc[postpaid_b2b_df['METRIC_NAME'].str.contains('B2B')]
postpaid_b2b_df = postpaid_b2b_df.loc[postpaid_b2b_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2b_df = postpaid_b2b_df.reset_index(drop=True)
postpaid_b2b_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202505,Postpaid,B2R020100,Postpaid Revenue B2B,2025-06-04 23:49:33,"547,336,917","547,336,917",0,0,0,0
1,202505,Postpaid,DB2R020100,Postpaid Revenue B2B : DTAC,2025-06-05 03:21:13,"201,194,460","201,194,460","201,194,460","200,111,432","200,111,432",0
2,202505,Postpaid,TB2R020100,Postpaid Revenue B2B : TMH,2025-06-04 23:49:33,"370,152,809","370,152,809",0,0,0,0


### TOL

In [30]:
''' TOL '''

v_product_grp = 'TOL'

tol_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_df = tol_df.loc[tol_df['TM_KEY_MTH']==v_tm_key_mth]
tol_df = tol_df.reset_index(drop=True)
tol_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202505,TOL,TB3R000100,TOL Revenue,2025-06-05 03:21:13,"1,618,217,396","1,614,210,491","1,614,210,491","1,614,210,491","1,614,210,491","4,006,904"


### TVS

In [31]:
''' TVS '''

v_product_grp = 'TVS'

tvs_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tvs_df = tvs_df.loc[tvs_df['TM_KEY_MTH']==v_tm_key_mth]
tvs_df = tvs_df.reset_index(drop=True)
tvs_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202505,TVS,TB4R000100,TVS Revenue,2025-06-05 03:21:13,"304,401,837","292,180,683","292,180,683","292,396,183","292,337,599","12,221,154"
